<a href="https://colab.research.google.com/github/EdiNel0407/us-ie-big-data-technologies/blob/main/postblock2/q3/postblock2_q3_pyspark_pagila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PostBlock2 Q3 — PySpark + PostgreSQL (pagila)

This notebook connects to a **PostgreSQL** database using **PySpark JDBC** and prints the schema of `public.customer` from the **pagila** database.

**Instructions**:
1. Run each cell top-to-bottom in **Google Colab**.
2. Edit the connection details (host, user, password) in the **Connection** cell.
3. After `df.printSchema()` runs, take a screenshot of the output cell for STEMLearn.
4. Commit this notebook to your private repo at `postblock2/q3/`.


## 1) Environment (Colab)

In [1]:
!pip -q install pyspark==3.5.1


## 2) Start Spark with the PostgreSQL JDBC driver

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName('Pagila_JDBC_Schema')
    .config('spark.jars.packages', 'org.postgresql:postgresql:42.7.3')
    .getOrCreate()
)
print('Spark version:', spark.version)


Spark version: 3.5.1


## 3) Connection (EDIT THESE)

In [3]:
PG_HOST = 'ep-winter-king-a8bvb2nf.eastus2.azure.neon.tech'   # e.g. 'db.example.com' or public IP
PG_PORT = '5432'
PG_DB   = 'pagila'
PG_USER = 'neondb_owner'
PG_PASS = 'npg_h4gJ1SuEWxae'
TABLE   = 'public.customer'

# If TLS is required by your server keep sslmode=require; otherwise you can remove the query string.
JDBC_URL = f'jdbc:postgresql://{PG_HOST}:{PG_PORT}/{PG_DB}?sslmode=require'
JDBC_DRIVER = 'org.postgresql.Driver'


## 4) Read via JDBC and print schema

In [4]:
df = (
    spark.read.format('jdbc')
    .option('url', JDBC_URL)
    .option('dbtable', TABLE)
    .option('user', PG_USER)
    .option('password', PG_PASS)
    .option('driver', JDBC_DRIVER)
    .load()
)
df.printSchema()


root
 |-- customer_id: integer (nullable = true)
 |-- store_id: short (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- address_id: short (nullable = true)
 |-- activebool: boolean (nullable = true)
 |-- create_date: date (nullable = true)
 |-- last_update: timestamp (nullable = true)
 |-- active: integer (nullable = true)



## 5) (Optional) Peek rows to verify connectivity

In [5]:
df.limit(5).show(truncate=False)


+-----------+--------+----------+---------+-----------------------+----------+----------+-----------+--------------------------+------+
|customer_id|store_id|first_name|last_name|email                  |address_id|activebool|create_date|last_update               |active|
+-----------+--------+----------+---------+-----------------------+----------+----------+-----------+--------------------------+------+
|1          |1       |Mary      |Smith    |mary.smith@example.com |1         |true      |2025-10-02 |2025-10-02 17:53:07.497835|1     |
|2          |1       |Patricia  |Johnson  |pat.johnson@example.com|2         |true      |2025-10-02 |2025-10-02 17:53:07.497835|1     |
+-----------+--------+----------+---------+-----------------------+----------+----------+-----------+--------------------------+------+



## 6) Save & Commit to GitHub (UI method)
In Colab: **File → Save a copy in GitHub**

• Select your private repo and branch (e.g., `main`).
• In the **File path** box, set: `postblock2/q3/postblock2_q3_pyspark_pagila.ipynb`
• Commit message: `PostBlock2 Q3: PySpark JDBC schema for pagila.customer`.

Alternatively, you can use Git CLI, but the UI is simplest.

## 7) Cleanup (optional)

In [ ]:
spark.stop()
print('Spark stopped.')


In [6]:
# Make sure `df` is the DataFrame you loaded from JDBC (public.customer)
# If your notebook kernel restarted, re-run the previous cells to recreate `df`.

# Create/replace a SQL view for Spark SQL queries
df.createOrReplaceTempView("customer")

# Spark SQL: count distinct, non-null email addresses
# (lower() treats EMAIL vs email as the same; filter out NULL/empty)
result = spark.sql("""
    SELECT COUNT(DISTINCT LOWER(TRIM(email))) AS unique_email_count
    FROM customer
    WHERE email IS NOT NULL AND LENGTH(TRIM(email)) > 0
""")

result.show()  # <-- Take the screenshot of this output for STEMLearn


+------------------+
|unique_email_count|
+------------------+
|                 2|
+------------------+

